In [ ]:
! pip install skforecast

# Modelo de Regresión para la Laguna de Torrevieja

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from scipy import stats

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Modelado y Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster

from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import  KNeighborsRegressor
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.ar_model import AutoReg

from skforecast.utils import load_forecaster

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras import optimizers,callbacks
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout, MultiHeadAttention, GlobalAveragePooling1D, Input

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# Calculo de metricas
# ==============================================================================
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn import metrics

# Configuración warnings
# ==============================================================================
import warnings
pd.options.mode.chained_assignment = None  # default='warn'
# warnings.filterwarnings('ignore')

## Preparación de los datos

In [ ]:
folder = ""
parametro = 'temperatura'
output=parametro
inputs = [
          'fecha', 'temperatura',
          'ambiente', 'nivel'
          ]
usecols = inputs.copy()

datos = pd.read_csv(folder + "lagoon_hourly_filled" + ".csv",sep=',', usecols=usecols)

# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%d/%m/%Y %H:%M')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('H')
datos = datos.sort_index()

datos.info()

datos = datos.iloc[:]

#datos.head()
datos

Limpieza de datos anómalos

In [ ]:
datos = datos.iloc[:]
datos_originales = datos.copy()


datos.info()


## Predecir T

In [ ]:
datos["t-1"] = datos["temperatura"].shift(1)
# datos["ta-12"] = datos["ambiente"].shift(12)
# datos["ta-24"] = datos["ambiente"].shift(24)
# datos["ta-36"] = datos["ambiente"].shift(36)
# datos["ta-48"] = datos["ambiente"].shift(48)

datos = datos.dropna()

datos.info()

## Preparar los datos de train y test

In [ ]:
entreno = 420 # LA SUMA ES PARA MOVERME EN LA FRANJA HORARIA DEL MISMO DÍA (ASCENDENTE O DESCENDENTE)
cola = 420 #SIEMPRE HACERLO DINÁMICO, QUE PODAMOS PASARLO POR EJEMPLO COMO ARGUMENTO

datos_dif = datos.iloc[:] #JUGAR CON LOS VALORES DE "X:Y" PARA MARCAR LA SERIE TEMPORAL QUE SE QUIERE
#datos_dif["temperatura"] = datos_dif["temperatura"].diff().dropna()
#datos_dif["t-1"] = datos_dif["t-1"].diff().dropna()
datos_dif = datos_dif.dropna()
datos_pred = datos_dif.iloc[:-entreno] # PARA TENER UNA COPIA DEL ORIGINAL Y MODIFICAR LA COPIA
predicciones = pd.DataFrame() #ESTO PARA CREAR UN DATAFRAME SOLO CON LAS PREDICCIONES, PARA PINTAR LUEGO

datos_dif.info()

In [ ]:
datos_pred.info()
datos_pred

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
datos_dif['temperatura'].plot(ax=ax, label='totales')
datos_pred['temperatura'].plot(ax=ax, label='entrenar')
ax.legend();

In [ ]:
from tensorflow.keras.optimizers import Adam
df = datos_dif.copy()

num_features = df.shape[1]

# interpolate missing values
df = df.interpolate(method='linear', axis=0)

# remove all nan values
df = df.dropna()

# Get the minimum and maximum values of 'eto'
min_et = df['temperatura'].min()
max_et = df['temperatura'].max()

df

In [ ]:
df

In [ ]:
# Normalize data
scaled_data = df[df.columns].values

scalers = {}
for column in df.columns:
    scalers[column] = MinMaxScaler(feature_range=(0, 1))
    scaled_data[:, df.columns.get_loc(column)] = scalers[column].fit_transform(scaled_data[:, df.columns.get_loc(column)].reshape(-1, 1)).flatten()

# Prepare input and output sequences
n_days = entreno  # Number of days in each sequence
X = []
y = []
for i in range(n_days, len(df) - n_days + 1):
    X.append(scaled_data[i-n_days:i])
    y.append(scaled_data[i:i+n_days, df.columns.get_loc('temperatura')])  # Output sequence with 7 days of Evapotranspiration


X = np.array(X)
y = np.array(y)

scaled_data

In [ ]:

# Define the callbacks
early_stopping = EarlyStopping(patience=50, monitor='val_loss', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
lr_scheduler = LearningRateScheduler(lambda epoch: 0.001 * 0.9 ** epoch)  # Example learning rate scheduler


# Define LSTM model
model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='tanh', input_shape=(n_days, X.shape[2])))
model.add(keras.layers.Dense(n_days))
optimizer = Adam(learning_rate=0.001, clipvalue=1.0)  # Clip gradients to prevent explosion
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.add(keras.layers.Dropout(0.2))


# Apply TimeSeriesSplit for cross-validation
mse_scores = []
mae_scores = []
r2_score = []
tscv = TimeSeriesSplit(n_splits=10, test_size=1)  # Assuming you want to split the data into 5 folds
for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, y_train, epochs=500, batch_size=24, validation_data=(X_test, y_test), callbacks=[early_stopping, lr_scheduler], verbose=0)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test)
    print(f"Fold {fold} - Test Loss: {loss}")

    # Make predictions
    predictions = model.predict(X_test)
    print(predictions)

    # Inverse transform the predictions and actual values
    inv_predictions = scalers['temperatura'].inverse_transform(predictions).flatten()
    inv_actual = scalers['temperatura'].inverse_transform(y_test).flatten()
    inv_input = scalers['temperatura'].inverse_transform(X_test[:, :, df.columns.get_loc('temperatura')].reshape(1, -1)).flatten()
    print(inv_predictions)
    # Calculate MSE
    mse = mean_squared_error(inv_actual, inv_predictions)
    mse_scores.append(mse)
    print(f"Fold {fold} - MSE: {mse}")

    # Calculate MAE
    mae = np.mean(np.abs(inv_actual - inv_predictions))
    mae_scores.append(mae)
    print(f"Fold {fold} - MAE: {mae}")

    #Caclular R2
    r2 = metrics.r2_score(inv_actual, inv_predictions)
    r2_score.append(r2)

    # Plot predictions, actual values, and input data
    plt.figure(figsize=(15, 7))
    plt.plot(np.arange(len(inv_input)), inv_input, label='Datos entrenamiento', linestyle='--')
    plt.plot(np.arange(len(inv_input), len(inv_input) + len(inv_actual)), inv_predictions, label='Predicción')
    plt.plot(np.arange(len(inv_input), len(inv_input) + len(inv_actual)), inv_actual, label='Original')

    # Configuración de los ejes y títulos
    plt.xlabel('Fecha', fontsize=14)
    plt.ylabel('Temperatura (ºC)', fontsize=14)
    plt.title(f'Fold {fold} - LSTM', fontsize=16)

    # Quitar los bordes superior y derecho
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Ajustar el tamaño de la leyenda
    plt.legend(fontsize=12)

    # Ajustar el tamaño de las etiquetas de los ejes
    ax.tick_params(axis='both', which='major', labelsize=12)

    # Configurar la cuadrícula
    ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey')

    # Guardar el gráfico en una carpeta con buena resolución
    plt.savefig(f"fold_{fold}_LSTM.png", dpi=300, bbox_inches='tight')

    # Mostrar el gráfico
    plt.show()

# Calculate average MSE across all splits
print(f"Average MSE: {np.mean(mse_scores)}")
print(f"Average MAE: {np.mean(mae_scores)}")
print(f"Average r2: {np.mean(r2_score)}")

In [ ]:
start_date = "2023-08-09 00:00"
end_date = "2023-09-13 00:00"
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

date_range

In [ ]:
len(date_range)

In [ ]:
len(inv_input)

In [ ]:
len(inv_predictions)

In [ ]:
len(inv_input) + len(inv_actual)

In [ ]:
# Plot predictions, actual values, and input data
plt.figure(figsize=(15, 7))
plt.plot(np.arange(len(inv_input)), inv_input, label='Datos entrenamiento', linestyle='--')
plt.plot(np.arange(len(inv_input), len(inv_input) + len(inv_actual)), inv_predictions, label='Predicción')
plt.plot(np.arange(len(inv_input), len(inv_input) + len(inv_actual)), inv_actual, label='Original')

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Experimento 4 - LSTM Fold {fold}  (420 horas)', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey')


ax.set_xticks(np.arange(len(date_range))[::168])  # Mostrar ticks cada 168 horas (una semana)
ax.set_xticklabels(date_range[::168].strftime('%Y-%m-%d %H:%M'))

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"fold_{fold}_LSTM.png", dpi=300, bbox_inches='tight')

# Mostrar el gráfico
plt.show()